# 使用 MindSpore 复现 bace 预测

## 1. 导入必要的库和模块

In [1]:
import pandas as pd
import mindspore
from mindspore import context

from data_pre import split_data
from train import run_training, run_testing

context.set_context(mode=context.PYNATIVE_MODE, device_target="CPU")

print('------------------------')

/usr/local/Ascend/ascend-toolkit/7.0.RC1/python/site-packages/tbe/tvm/contrib/ccec.py:766: DeprecationWarning: invalid escape sequence \L
  if not dirpath.find("AppData\Local\Temp"):
/usr/local/Ascend/ascend-toolkit/latest/python/site-packages/tbe/dsl/classifier/transdata/transdata_classifier.py:222: DeprecationWarning: invalid escape sequence \B
  """
/usr/local/Ascend/ascend-toolkit/latest/python/site-packages/tbe/dsl/unify_schedule/vector/transdata/common/graph/transdata_graph_info.py:140: DeprecationWarning: invalid escape sequence \c
  """
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, 

------------------------


## 2. 定义实验参数

我们将所有可调参数集中定义在这里，方便修改和管理。

In [12]:
class Args:
    # --- 数据和路径参数 ---
    data_path = 'data/bace.csv'
    smiles_column = 'mol'
    target_column = 'Class'
    save_dir = 'bace_save_model'
    plot_save_dir = 'bace_plot'

    # --- 数据集划分参数 ---
    split_type = 'random' # 可选 'random' 或 'scaffold'
    split_sizes = [0.8, 0.1, 0.1]

    # --- 模型超参数 ---
    hidden_size = 300
    depth = 3
    dropout = 0.1

    # --- 训练超参数 ---
    epochs = 30
    batch_size = 32
    learning_rate = 1e-4

args = Args()
print('------------------------')

------------------------


## 3. 执行主流程

现在，我们按顺序执行数据加载、划分、训练和测试。

In [13]:
# ---加载数据 ---
try:
    df = pd.read_csv(args.data_path)
    print(f"Successfully loaded data from '{args.data_path}'. Total molecules: {len(df)}")
    display(df.head())
except FileNotFoundError:
    print(f"Error: Data file not found at '{args.data_path}'")
    print("Please create a CSV file with 'SMILES' and 'ACTIVITY' columns, and update the `data_path` argument.")

if 'df' in locals():
    # ---划分数据集 ---
    train_data, val_data, test_data = split_data(
        df=df, 
        smiles_column=args.smiles_column, 
        target_column=args.target_column, 
        split_type=args.split_type, 
        split_sizes=args.split_sizes
    )
    
print('------------------------')

Successfully loaded data from 'data/bace.csv'. Total molecules: 1513


,mol,CID,Class,Model,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,PEOE6 (PEOE6),PEOE7 (PEOE7),PEOE8 (PEOE8),PEOE9 (PEOE9),PEOE10 (PEOE10),PEOE11 (PEOE11),PEOE12 (PEOE12),PEOE13 (PEOE13),PEOE14 (PEOE14),canvasUID
0,O1CC[C@@H](NC(=O)[C@@H](Cc2cc3cc(ccc3nc2N)-c2c...,BACE_1,1,Train,NaN,NaN,NaN,NaN,NaN,NaN,...,53.205711,78.640335,226.85541,107.43491,37.133846,0.000000,7.980170,0.0,0.000000,1
1,Fc1cc(cc(F)c1)C[C@H](NC(=O)[C@@H](N1CC[C@](NC(...,BACE_2,1,Train,NaN,NaN,NaN,NaN,NaN,NaN,...,73.817162,47.171600,365.67694,174.07675,34.923889,7.980170,24.148668,0.0,24.663788,2
2,S1(=O)(=O)N(c2cc(cc3c2n(cc3CC)CC1)C(=O)N[C@H](...,BACE_3,1,Train,NaN,NaN,NaN,NaN,NaN,NaN,...,70.365707,47.941147,192.40652,255.75255,23.654478,0.230159,15.879790,0.0,24.663788,3
3,S1(=O)(=O)C[C@@H](Cc2cc(O[C@H](COCC)C(F)(F)F)c...,BACE_4,1,Train,NaN,NaN,NaN,NaN,NaN,NaN,...,56.657166,37.954151,194.35304,202.76335,36.498634,0.980913,8.188327,0.0,26.385181,4
4,S1(=O)(=O)N(c2cc(cc3c2n(cc3CC)CC1)C(=O)N[C@H](...,BACE_5,1,Train,NaN,NaN,NaN,NaN,NaN,NaN,...,78.945702,39.361153,179.71288,220.46130,23.654478,0.230159,15.879790,0.0,26.100143,5


Splitting data with method: 'random'
Data split sizes: Train=1209, Validation=152, Test=152
------------------------


## 4. 训练并获取最佳模型路径

In [14]:
best_model_path = run_training(args, train_data, val_data)
print('------------------------')


Starting training...


Epoch 1/30: 100%|██████████| 38/38 [00:09<00:00,  4.19it/s, loss=0.6456]


Epoch 01 | Train Loss: 0.6863 | Val AUC: 0.5997
  -> New best model saved with AUC: 0.5997


Epoch 2/30: 100%|██████████| 38/38 [00:08<00:00,  4.28it/s, loss=0.7136]


Epoch 02 | Train Loss: 0.6837 | Val AUC: 0.6119
  -> New best model saved with AUC: 0.6119


...

Epoch 28 | Train Loss: 0.6246 | Val AUC: 0.7520


Epoch 29/30: 100%|██████████| 38/38 [00:09<00:00,  3.89it/s, loss=0.5329]


Epoch 29 | Train Loss: 0.6226 | Val AUC: 0.7585
  -> New best model saved with AUC: 0.7585


Epoch 30/30: 100%|██████████| 38/38 [00:09<00:00,  3.89it/s, loss=0.6339]


Epoch 30 | Train Loss: 0.6223 | Val AUC: 0.7585
  -> New best model saved with AUC: 0.7585
Training finished.
------------------------


## 5. 使用最佳模型进行测试

In [11]:
run_testing(args, test_data, best_model_path)
print('------------------------')


Starting final testing...
Loaded best model from: bace_save_model/best_model.ckpt


Testing: 100%|██████████| 5/5 [00:00<00:00, 12.45it/s]



===== Final Test Results =====
  Test Set AUC:                 0.5309
  Test Set Accuracy:            0.5592
  Test Set Average Precision:   0.5253
Evaluation summary plot saved to bace_plot/testset_performance_evaluation_curves.png
------------------------
